<a href="https://colab.research.google.com/github/joacoxmdb/couples/blob/main/couples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mtcnn
!pip install pillow


In [2]:
import os
import cv2
import numpy as np
from mtcnn import MTCNN
from PIL import Image


In [3]:
# Inicializar el detector de rostros
detector = MTCNN()

def detect_and_align_face(image_path):
    # Cargar la imagen
    img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

    # Detectar los rostros en la imagen
    detections = detector.detect_faces(img)

    # Si no se detecta ningún rostro, devolver None
    if len(detections) == 0:
        return None

    # Tomar el primer rostro detectado
    face_data = detections[0]['box']
    x, y, w, h = face_data

    # Recortar y alinear el rostro
    face = img[y:y+h, x:x+w]

    # Redimensionar a 160x160 píxeles
    face_image = Image.fromarray(face)
    face_image = face_image.resize((160, 160))

    return face_image


In [4]:
# Configurar directorios
input_dir = '/content/dataset'  # Cambia esto a la ruta de tus imágenes
output_dir = '/content/processed'  # Ruta donde se guardarán las imágenes procesadas

# Crear las carpetas de salida si no existen
os.makedirs(output_dir, exist_ok=True)
os.makedirs(f"{output_dir}/yo", exist_ok=True)
os.makedirs(f"{output_dir}/novia", exist_ok=True)
os.makedirs(f"{output_dir}/juntos", exist_ok=True)

# Función para procesar todas las imágenes en una carpeta
def process_images(input_folder, output_folder):
    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        processed_face = detect_and_align_face(image_path)

        if processed_face:
            processed_face.save(os.path.join(output_folder, filename))
        else:
            print(f"No se detectó rostro en: {filename}")

# Procesar cada carpeta
process_images(f"{input_dir}/yo", f"{output_dir}/yo")
process_images(f"{input_dir}/novia", f"{output_dir}/novia")
process_images(f"{input_dir}/juntos", f"{output_dir}/juntos")


No se detectó rostro en: foto13.jpg
No se detectó rostro en: foto28.JPG
No se detectó rostro en: foto36.jpeg
No se detectó rostro en: foto1.jpg
No se detectó rostro en: foto14.PNG
No se detectó rostro en: foto43.jpeg
No se detectó rostro en: foto32.jpeg
No se detectó rostro en: foto50.jpeg


In [5]:
!pip install tensorflow

In [6]:
!pip install facenet-pytorch


In [7]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch

# Cargar el modelo para detección de rostros
mtcnn = MTCNN(keep_all=True, device='cuda' if torch.cuda.is_available() else 'cpu')

# Cargar el modelo FaceNet
facenet_model = InceptionResnetV1(pretrained='vggface2').eval()
print("Modelo FaceNet cargado con éxito")


  0%|          | 0.00/107M [00:00<?, ?B/s]

Modelo FaceNet cargado con éxito


In [15]:
import os
import pickle
import numpy as np
import torch
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1

# Cargar el modelo para detección de rostros
mtcnn = MTCNN(keep_all=True, device='cuda' if torch.cuda.is_available() else 'cpu')

# Cargar el modelo FaceNet
facenet_model = InceptionResnetV1(pretrained='vggface2').eval()

# Función para obtener embeddings
def get_embedding(model, face_image):
    # Convertir la imagen a tensor
    face_tensor = torch.tensor(np.asarray(face_image)).permute(2, 0, 1).float()  # Cambiar el orden a (C, H, W)

    # Normalizar la imagen
    face_tensor = (face_tensor - 127.5) / 127.5  # Normalización entre -1 y 1

    # Añadir dimensión para el batch
    face_tensor = face_tensor.unsqueeze(0)  # Dimensión (1, C, H, W)

    # Obtener el embedding
    with torch.no_grad():
        embedding = model(face_tensor)

    return embedding[0].numpy()  # Convertir el tensor a numpy array

# Función para crear embeddings y guardarlos
def create_embeddings(input_folder, label):
    embeddings = []
    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        face_image = Image.open(image_path)
        embedding = get_embedding(facenet_model, face_image)
        embeddings.append(embedding)
        print(f"Embedding generado para: {filename}")
    return embeddings

# Cargar el directorio procesado (asegúrate de que el directorio existe)
processed_dir = '/content/processed'  # Actualiza esta ruta

# Crear embeddings para cada carpeta
embeddings = {
    'yo': create_embeddings(f"{processed_dir}/yo", 'yo'),
    'novia': create_embeddings(f"{processed_dir}/novia", 'novia'),
    'juntos': create_embeddings(f"{processed_dir}/juntos", 'juntos')
}

# Guardar embeddings en un archivo
with open('embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

print("Embeddings generados y guardados con éxito")


Embedding generado para: foto47.jpeg
Embedding generado para: foto4.jpg
Embedding generado para: foto27.JPG
Embedding generado para: foto22.jpg
Embedding generado para: foto45.jpeg
Embedding generado para: foto6.jpg
Embedding generado para: foto12.jpg
Embedding generado para: foto2.jpg
Embedding generado para: foto20.jpg
Embedding generado para: foto21.JPG
Embedding generado para: foto41.jpeg
Embedding generado para: foto8.jpg
Embedding generado para: foto26.PNG
Embedding generado para: foto7.jpg
Embedding generado para: foto15.PNG
Embedding generado para: foto3.jpg
Embedding generado para: foto11.jpg
Embedding generado para: foto49.jpeg
Embedding generado para: foto9.jpg
Embedding generado para: foto33.jpeg
Embedding generado para: foto17.jpg
Embedding generado para: foto25.jpg
Embedding generado para: foto53.jpeg
Embedding generado para: foto24.jpg
Embedding generado para: foto52.jpeg
Embedding generado para: foto35.jpeg
Embedding generado para: foto42.jpeg
Embedding generado para: f

In [16]:
# Cargar embeddings desde el archivo
with open('embeddings.pkl', 'rb') as f:
    embeddings = pickle.load(f)

print("Embeddings cargados con éxito")


Embeddings cargados con éxito


In [17]:
from scipy.spatial.distance import cosine

# Definir umbral de distancia para considerar un rostro como coincidente
threshold = 0.6  # Este valor puede necesitar ajuste

def is_matching(embedding, known_embeddings):
    for label, known_embeds in known_embeddings.items():
        for known_embedding in known_embeds:
            # Calcular la distancia entre el embedding y el embedding conocido
            dist = cosine(embedding, known_embedding)
            if dist < threshold:
                return label  # Devuelve la etiqueta del rostro coincidente
    return None  # No se encontró coincidencia


In [22]:
def process_and_compare_image(image_path):
    # Cargar y procesar la nueva imagen
    face_image = Image.open(image_path)
    embedding = get_embedding(facenet_model, face_image)  # Obtener el embedding de la nueva imagen

    # Comparar el embedding con los embeddings conocidos
    match = is_matching(embedding, embeddings)

    if match:
        print(f"La imagen contiene: {match}")
    else:
        print("No se encontró coincidencia en la imagen.")


In [42]:
new_image_path = '/content/test/7F1A7801-DD20-4715-82C5-7670E234F54D.JPG'  # Cambia esta ruta a tu nueva imagen
process_and_compare_image(new_image_path)


No se encontró coincidencia en la imagen.


In [25]:
!pip install streamlit
!pip install localtunnel  # Para usar LocalTunnel, si decides usarlo


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 4.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement localtunnel (from versions: none)
ERROR: No matching distribution found for localtunnel


In [26]:
%%writefile app.py
import streamlit as st
import os
import pickle
import numpy as np
import torch
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1
from scipy.spatial.distance import cosine

# Cargar modelos
mtcnn = MTCNN(keep_all=True, device='cuda' if torch.cuda.is_available() else 'cpu')
facenet_model = InceptionResnetV1(pretrained='vggface2').eval()

# Cargar embeddings guardados
with open('embeddings.pkl', 'rb') as f:
    embeddings = pickle.load(f)

# Funciones de la aplicación
def get_embedding(model, face_image):
    face_tensor = torch.tensor(np.asarray(face_image)).permute(2, 0, 1).float()
    face_tensor = (face_tensor - 127.5) / 127.5
    face_tensor = face_tensor.unsqueeze(0)
    with torch.no_grad():
        embedding = model(face_tensor)
    return embedding[0].numpy()

def is_matching(embedding, known_embeddings, threshold=0.6):
    for label, known_embeds in known_embeddings.items():
        for known_embedding in known_embeds:
            dist = cosine(embedding, known_embedding)
            if dist < threshold:
                return label
    return None

# Configuración de Streamlit
st.title("Verificador de Rostros")
st.write("Carga una imagen para verificar si contiene a ti o a tu novia.")

uploaded_file = st.file_uploader("Selecciona una imagen", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Imagen cargada', use_column_width=True)

    embedding = get_embedding(facenet_model, image)
    match = is_matching(embedding, embeddings)

    if match:
        st.success(f"La imagen contiene: {match}")
    else:
        st.error("No se encontró coincidencia en la imagen.")


Writing app.py


In [28]:
!streamlit run app.py & npx localtunnel --port 8501




Need to install the following packages:
  localtunnel@2.0.2
Ok to proceed? (y) 
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.222.121:8501

  Stopping...
^C


In [29]:
!pip install pyngrok


In [39]:
!streamlit run couples.ipynb


Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Streamlit requires raw Python (.py) files, not .ipynb.
For more information, please see https://docs.streamlit.io


In [40]:
!ngrok authtoken 2oMn74uwzDqtNxKy0UnYIXJw1ek_7S1r5ra3UqYogWWC7Dbt4



Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [41]:
from pyngrok import ngrok

# Finalizar cualquier túnel que ya esté activo
ngrok.kill()

# Iniciar un nuevo túnel
public_url = ngrok.connect("8501")  # No es necesario especificar el puerto, sólo se necesita el nombre del puerto
public_url



<NgrokTunnel: "https://4428-34-125-222-121.ngrok-free.app" -> "http://localhost:8501">